<a href="https://colab.research.google.com/github/Linerah/predict-delirium-elderly/blob/main/SelfLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

!gcloud projects list

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='clinical-entity-extraction')

# List all tables in the dataset.
dataset_ref = client.get_dataset('physionet-data.mimiciii_clinical')
dataset_notes_ref = client.get_dataset('physionet-data.mimiciii_notes')
tables = list(client.list_tables(dataset_ref))
tables_notes = list(client.list_tables(dataset_notes_ref))

print("Tables:")
for table in tables:
    table_ref = dataset_ref.table(table.table_id)
    table = client.get_table(table_ref)
    print(f"  {table.table_id}: {table.num_rows} rows")
for table in tables_notes:
    table_ref = dataset_notes_ref.table(table.table_id)
    table = client.get_table(table_ref)
    print(f"  {table.table_id}: {table.num_rows} rows")


Authenticated
PROJECT_ID                  NAME                        PROJECT_NUMBER
clinical-entity-extraction  clinical-entity-extraction  321960627270
divine-tempo-450522-t7      My First Project            417906181427
Tables:
  admissions: 58976 rows
  callout: 34499 rows
  caregivers: 7567 rows
  chartevents: 330712483 rows
  cptevents: 573146 rows
  d_cpt: 134 rows
  d_icd_diagnoses: 14567 rows
  d_icd_procedures: 3882 rows
  d_items: 12487 rows
  d_labitems: 753 rows
  datetimeevents: 4485937 rows
  diagnoses_icd: 651047 rows
  drgcodes: 125557 rows
  icustays: 61532 rows
  inputevents_cv: 17527935 rows
  inputevents_mv: 3618991 rows
  labevents: 27854055 rows
  microbiologyevents: 631726 rows
  outputevents: 4349218 rows
  patients: 46520 rows
  prescriptions: 4156450 rows
  procedureevents_mv: 258066 rows
  procedures_icd: 240095 rows
  services: 73343 rows
  transfers: 261897 rows
  noteevents: 2083180 rows


In [2]:
!pip install pandas==2.0.3
!pip install itables==1.3.0
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import pandas as pd

  Using cached itables-1.3.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached itables-1.3.0-py3-none-any.whl (193 kB)
Using cached jedi-0.19.2-py2.py3-none-any.whl (1.6 MB)


In [3]:
# Define the query
query = """
WITH cognitive_impairments AS (
    SELECT
        di.subject_id,
        di.hadm_id,
        CASE
            WHEN di.icd9_code IN ('2900', '2901', '2902', '2903', '2941', '3310') THEN 'Dementia'
            WHEN di.icd9_code IN ('33182') THEN 'Alzheimers'
            ELSE 'Other Cognitive Impairment'
        END AS cognitive_impairment_type
    FROM
        `physionet-data.mimiciii_clinical.diagnoses_icd` AS di
    WHERE
        di.icd9_code IN ('2900', '2901', '2902', '2903', '2941', '3310', '33182')
),
comorbidities AS (
    SELECT
        di.subject_id,
        di.hadm_id,
        MAX(CASE WHEN di.icd9_code IN ('99591', '99592', '78552') THEN 1 ELSE 0 END) AS sepsis,
        MAX(CASE WHEN di.icd9_code IN ('4280', '4281', '4282', '4283', '4284', '4285', '4286', '4289') THEN 1 ELSE 0 END) AS heart_failure,
        MAX(CASE WHEN di.icd9_code IN ('43491', '43411', '43401', '436') THEN 1 ELSE 0 END) AS stroke
    FROM
        `physionet-data.mimiciii_clinical.diagnoses_icd` AS di
    GROUP BY
        di.subject_id, di.hadm_id
),
icu_admissions AS (
    SELECT
        icu.subject_id,
        icu.hadm_id,
        1 AS icu_admission
    FROM
        `physionet-data.mimiciii_clinical.icustays` AS icu
    GROUP BY
        icu.subject_id, icu.hadm_id
),
surgeries AS (
    SELECT
        pr.subject_id,
        pr.hadm_id,
        MAX(CASE WHEN pr.icd9_code IN ('3961', '3962', '8154', '8155', '8151', '8152') THEN 1 ELSE 0 END) AS major_surgery
    FROM
        `physionet-data.mimiciii_clinical.procedures_icd` AS pr
    GROUP BY
        pr.subject_id, pr.hadm_id
),
delirium_diagnosis AS (
    SELECT
        di.subject_id,
        di.hadm_id,
        MAX(CASE WHEN di.icd9_code IN ('2930', '2931', '78009', '78002', '7801', '78097', '29011', '2903', '3102') THEN 1 ELSE 0 END) AS delirium
    FROM
        `physionet-data.mimiciii_clinical.diagnoses_icd` AS di
    GROUP BY
        di.subject_id, di.hadm_id
),
medication_usage AS (
    SELECT
        a.subject_id,
        a.hadm_id,
        MAX(CASE
            WHEN LOWER(d.drug) LIKE '%lorazepam%'
              OR LOWER(d.drug) LIKE '%midazolam%'
              OR LOWER(d.drug) LIKE '%diazepam%'
            THEN 1 ELSE 0
        END) AS sedatives,
        MAX(CASE
            WHEN LOWER(d.drug) LIKE '%diphenhydramine%'
              OR LOWER(d.drug) LIKE '%atropine%'
              OR LOWER(d.drug) LIKE '%scopolamine%'
            THEN 1 ELSE 0
        END) AS anticholinergics,
        MAX(CASE
            WHEN LOWER(d.drug) LIKE '%morphine%'
              OR LOWER(d.drug) LIKE '%fentanyl%'
              OR LOWER(d.drug) LIKE '%hydromorphone%'
              OR LOWER(d.drug) LIKE '%oxycodone%'
            THEN 1 ELSE 0
        END) AS opioids,
        COUNT(DISTINCT d.drug) AS total_medications
    FROM
        `physionet-data.mimiciii_clinical.prescriptions` AS d
    JOIN
        `physionet-data.mimiciii_clinical.admissions` AS a
    ON
        d.subject_id = a.subject_id AND d.hadm_id = a.hadm_id
    GROUP BY
        a.subject_id, a.hadm_id
),
vital_signs AS (
    SELECT
        ce.subject_id,
        ce.hadm_id,
        MAX(CASE
            WHEN ce.itemid IN (211, 220045) AND ce.valuenum > 38 THEN 1 ELSE 0
        END) AS fever,
        MAX(CASE
            WHEN ce.itemid IN (51, 220050) AND ce.valuenum < 90 THEN 1 ELSE 0
        END) AS hypotension
    FROM
        `physionet-data.mimiciii_clinical.chartevents` AS ce
    WHERE
        ce.itemid IN (211, 220045, 51, 220050) -- 211 and 220045: Temperature; 51 and 220050: Mean Arterial Pressure
    GROUP BY
        ce.subject_id, ce.hadm_id
),
lab_values AS (
    SELECT
        le.subject_id,
        le.hadm_id,
        MAX(CASE
            WHEN le.itemid IN (50983, 50984) AND le.valuenum < 135 THEN 1 ELSE 0
        END) AS hyponatremia,
        MAX(CASE
            WHEN le.itemid IN (50971) AND le.valuenum > 5 THEN 1 ELSE 0
        END) AS hyperkalemia,
        MAX(CASE
            WHEN le.itemid IN (51265) AND le.valuenum > 10 THEN 1 ELSE 0
        END) AS infection
    FROM
        `physionet-data.mimiciii_clinical.labevents` AS le
    WHERE
        le.itemid IN (50983, 50984, 50971, 51265) -- Sodium (50983, 50984), Potassium (50971), WBC (51265)
    GROUP BY
        le.subject_id, le.hadm_id
)
SELECT
    p.subject_id,
    p.gender,
    p.dob,
    a.admittime,
    a.dischtime,
    a.ethnicity,
    ci.cognitive_impairment_type,
    p.expire_flag AS is_deceased,
    ROUND(DATE_DIFF(a.dischtime, a.admittime, DAY), 2) AS length_of_stay,
    c.sepsis,
    c.heart_failure,
    c.stroke,
    COALESCE(icu.icu_admission, 0) AS icu_admission,
    COALESCE(s.major_surgery, 0) AS major_surgery,
    COALESCE(d.delirium, 0) AS delirium,
    COALESCE(mu.sedatives, 0) AS sedatives,
    COALESCE(mu.anticholinergics, 0) AS anticholinergics,
    COALESCE(mu.opioids, 0) AS opioids,
    COALESCE(mu.total_medications, 0) AS total_medications,
    COALESCE(vs.fever, 0) AS fever,
    COALESCE(vs.hypotension, 0) AS hypotension,
    COALESCE(lv.hyponatremia, 0) AS hyponatremia,
    COALESCE(lv.hyperkalemia, 0) AS hyperkalemia,
    COALESCE(lv.infection, 0) AS infection
FROM
    `physionet-data.mimiciii_clinical.patients` AS p
JOIN
    `physionet-data.mimiciii_clinical.admissions` AS a
ON
    p.subject_id = a.subject_id
LEFT JOIN
    cognitive_impairments AS ci
ON
    a.hadm_id = ci.hadm_id
LEFT JOIN
    comorbidities AS c
ON
    a.hadm_id = c.hadm_id
LEFT JOIN
    icu_admissions AS icu
ON
    a.hadm_id = icu.hadm_id
LEFT JOIN
    surgeries AS s
ON
    a.hadm_id = s.hadm_id
LEFT JOIN
    delirium_diagnosis AS d
ON
    a.hadm_id = d.hadm_id
LEFT JOIN
    medication_usage AS mu
ON
    a.hadm_id = mu.hadm_id
LEFT JOIN
    vital_signs AS vs
ON
    a.hadm_id = vs.hadm_id
LEFT JOIN
    lab_values AS lv
ON
    a.hadm_id = lv.hadm_id
ORDER BY
    p.dob DESC;

"""


# Run the query
query_job = client.query(query)

# Convert to DataFrame
df = query_job.to_dataframe()

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: d5c38b96-44e3-4fb8-b8a9-7276cbb3cb29


In [ ]:
df

TypeError: Object of type Timestamp is not JSON serializable

       subject_id gender        dob           admittime           dischtime  \
0            8982      M 2201-07-24 2201-07-24 17:44:00 2201-08-04 19:43:00   
1           25913      M 2201-07-17 2201-07-17 17:23:00 2201-07-19 19:00:00   
2           13243      M 2201-07-16 2201-07-16 14:22:00 2201-07-21 11:45:00   
3           24254      M 2201-06-29 2201-06-29 18:21:00 2201-07-03 15:15:00   
4           11400      M 2201-06-25 2201-06-25 00:07:00 2201-06-27 11:26:00   
...           ...    ...        ...                 ...                 ...   
58974       62884      F 1800-07-16 2100-07-16 19:48:00 2100-08-12 18:05:00   
58975       31288      F 1800-07-16 2100-07-16 11:29:00 2100-07-22 18:00:00   
58976       31585      M 1800-07-02 2100-07-02 19:28:00 2100-07-07 18:05:00   
58977       31585      M 1800-07-02 2100-10-06 16:47:00 2100-10-13 16:30:00   
58978       31585      M 1800-07-02 2100-12-29 00:19:00 2101-01-07 17:53:00   

             ethnicity cognitive_impairment_type  i

In [ ]:
from datetime import datetime
import pandas as pd

# Convert 'dob' and 'admittime' columns to datetime objects
df["dob"] = pd.to_datetime(df["dob"])
df["admittime"] = pd.to_datetime(df["admittime"])

# Define a function to calculate age at admission
def calculate_age_at_admission(dob, admittime):
    return (admittime - dob).days // 365

# Apply the function to calculate the age
df["age_at_admission"] = df.apply(lambda row: calculate_age_at_admission(row["dob"], row["admittime"]), axis=1)

# Filter out rows with unrealistic ages (e.g., age >= 120)
df = df[df["age_at_admission"] < 120]

# Optional: Filter for older adults (e.g., age >= 65)
df = df[df["age_at_admission"] >= 65] # 65 or more is clasified as elderly

# Display the first few rows of the filtered DataFrame
print(df.head())

      subject_id gender        dob           admittime           dischtime  \
7716       15025      M 2141-03-05 2207-06-16 22:12:00 2207-06-20 15:29:00   
8227       25313      M 2139-04-15 2205-06-18 13:00:00 2205-06-21 14:50:00   
8773        5199      M 2137-07-10 2204-04-08 08:15:00 2204-04-13 17:40:00   
8774        5199      M 2137-07-10 2204-09-21 23:12:00 2204-10-14 15:46:00   
8852       42763      F 2137-04-12 2203-10-19 10:00:00 2203-10-27 14:38:00   

                   ethnicity cognitive_impairment_type  is_deceased  \
7716                   WHITE                      None            0   
8227                   WHITE                      None            0   
8773  BLACK/AFRICAN AMERICAN                      None            0   
8774  BLACK/AFRICAN AMERICAN                      None            0   
8852                   WHITE                      None            0   

      length_of_stay  sepsis  ...  sedatives  anticholinergics  opioids  \
7716             4.0       0 

In [ ]:
# Define age groups
bins = [65, 74, 84, 120]  # Adjust the bins as needed
labels = ["65-74", "75-84", "85+"]

# Create the age_group column
df["age_group"] = pd.cut(df["age_at_admission"], bins=bins, labels=labels, right=True)

In [ ]:
# # Replace None or NaN in cognitive_impairment_type with "None"
df["cognitive_impairment_type"] = df["cognitive_impairment_type"].fillna("None")

In [ ]:
df

TypeError: Object of type Timestamp is not JSON serializable

       subject_id gender        dob           admittime           dischtime  \
7716        15025      M 2141-03-05 2207-06-16 22:12:00 2207-06-20 15:29:00   
8227        25313      M 2139-04-15 2205-06-18 13:00:00 2205-06-21 14:50:00   
8773         5199      M 2137-07-10 2204-04-08 08:15:00 2204-04-13 17:40:00   
8774         5199      M 2137-07-10 2204-09-21 23:12:00 2204-10-14 15:46:00   
8852        42763      F 2137-04-12 2203-10-19 10:00:00 2203-10-27 14:38:00   
...           ...    ...        ...                 ...                 ...   
56357       19777      F 2012-11-25 2100-09-24 19:32:00 2100-09-29 06:23:00   
56358        9051      F 2012-11-21 2101-01-14 19:34:00 2101-01-19 13:15:00   
56359       83922      F 2012-10-25 2101-02-19 14:13:00 2101-02-24 17:30:00   
56360       70816      M 2012-08-06 2101-01-06 02:32:00 2101-01-14 17:00:00   
56361       71915      F 2012-07-16 2101-03-08 04:57:00 2101-03-11 14:08:00   

                    ethnicity cognitive_impairment_

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming the filtered DataFrame `df` is already created
# df now includes only patients aged 65+

# 1. Distribution of Age at Admission by Cognitive Impairment Type
fig1 = px.histogram(
    df,
    x="age_at_admission",
    color="cognitive_impairment_type",
    barmode="overlay",
    nbins=20,
    marginal="box",
    color_discrete_sequence=px.colors.qualitative.Set3,
    title="Distribution of Age at Admission (65+) by Cognitive Impairment Type"
)
fig1.update_layout(xaxis_title="Age at Admission", yaxis_title="Frequency")

# 2. Total Admissions by Ethnicity and Cognitive Impairment Type (Stacked Bar Chart)
ethnicity_impairment = df.groupby(["ethnicity", "cognitive_impairment_type"]).size().reset_index(name="count")
fig2 = px.bar(
    ethnicity_impairment,
    x="ethnicity",
    y="count",
    color="cognitive_impairment_type",
    text="count",
    title="Total Admissions (65+) by Ethnicity and Cognitive Impairment Type",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig2.update_layout(xaxis_title="Ethnicity", yaxis_title="Total Admissions", xaxis_tickangle=45)

# 3. Gender Distribution by Cognitive Impairment Type (Pie Chart)
gender_impairment = df.groupby(["gender", "cognitive_impairment_type"]).size().reset_index(name="count")
fig3 = px.sunburst(
    gender_impairment,
    path=["gender", "cognitive_impairment_type"],
    values="count",
    title="Gender Distribution (65+) by Cognitive Impairment Type",
    color="count",
    color_continuous_scale=px.colors.sequential.RdBu
)

# 4. Age at Admission vs. Cognitive Impairment Type (Box Plot)
fig4 = px.box(
    df,
    x="cognitive_impairment_type",
    y="age_at_admission",
    color="cognitive_impairment_type",
    title="Age at Admission (65+) by Cognitive Impairment Type",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig4.update_layout(xaxis_title="Cognitive Impairment Type", yaxis_title="Age at Admission")

# 5. Deceased Status by Cognitive Impairment Type and Age Group (Stacked Bar Chart)
age_group_deceased = df.groupby(["age_group", "cognitive_impairment_type", "is_deceased"]).size().reset_index(name="count")

# Create the bar chart with separate y-axes
fig5 = px.bar(
    age_group_deceased,
    x="age_group",
    y="count",
    color="is_deceased",
    facet_col="cognitive_impairment_type",
    barmode="stack",
    title="Deceased Status (65+) by Age Group and Cognitive Impairment Type",
    color_discrete_sequence=["green", "red"]
)

# Update layout to use separate y-axes for each facet
fig5.update_layout(
    yaxis_title="Count",
    yaxis=dict(matches=None),  # Disable matching y-axes
    yaxis2=dict(matches=None),
    yaxis3=dict(matches=None),
    facet_col_spacing=0.05  # Adjust spacing between facets
)
# Display all figures in a dashboard-like format
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()


In [ ]:

fig_los = px.box(
    df,
    x="delirium",
    y="length_of_stay",
    color="delirium",
    title="Length of Stay by Delirium Diagnosis",
    labels={"delirium": "Delirium (0 = No, 1 = Yes)", "length_of_stay": "Length of Stay (Days)"}
)
fig_los.show()


In [ ]:
comorbidities = df[["sepsis", "heart_failure", "stroke", "delirium"]].melt(id_vars="delirium", var_name="Comorbidity", value_name="Presence")
comorbidity_counts = comorbidities.groupby(["Comorbidity", "Presence", "delirium"]).size().reset_index(name="count")
fig_comorbidities = px.bar(
    comorbidity_counts,
    x="Comorbidity",
    y="count",
    color="Presence",
    facet_col="delirium",
    title="Comorbidities by Delirium Diagnosis",
    barmode="stack"
)
fig_comorbidities.show()
print(df[df["delirium"] == 1].shape[0])

1231


In [ ]:
icu_counts = df.groupby(["icu_admission", "delirium"]).size().reset_index(name="count")
fig_icu = px.bar(
    icu_counts,
    x="icu_admission",
    y="count",
    color="delirium",
    title="ICU Admission by Delirium Diagnosis",
    labels={"icu_admission": "ICU Admission (0 = No, 1 = Yes)", "count": "Number of Patients"}
)
fig_icu.show()
print(df[df["delirium"] == 1].shape[0])

1231


In [ ]:
# Cleaning the data
import numpy as np

# Ensure the dataset is sorted so the admissions for each patient are in chronological order:
df.sort_values(by=['subject_id', 'admittime'], inplace=True)
df

TypeError: Object of type Timestamp is not JSON serializable

       subject_id gender        dob           admittime           dischtime  \
55621           3      M 2025-04-11 2101-10-20 19:08:00 2101-10-31 13:58:00   
21558           6      F 2109-06-21 2175-05-30 07:15:00 2175-06-15 16:00:00   
54493          12      M 2032-03-24 2104-08-07 10:15:00 2104-08-20 02:57:00   
22728          20      F 2107-06-13 2183-04-28 09:45:00 2183-05-03 14:45:00   
50564          21      M 2047-04-04 2134-09-11 12:17:00 2134-09-24 16:15:00   
...           ...    ...        ...                 ...                 ...   
30779       99982      M 2091-10-02 2157-01-05 17:27:00 2157-01-12 13:00:00   
30781       99982      M 2091-10-02 2157-02-16 17:31:00 2157-02-22 20:36:00   
18602       99983      M 2114-09-29 2193-04-26 11:35:00 2193-04-29 13:30:00   
37015       99992      F 2078-10-17 2144-07-25 18:03:00 2144-07-28 17:56:00   
46077       99995      F 2058-05-29 2147-02-08 08:00:00 2147-02-11 13:15:00   

                   ethnicity cognitive_impairment_t

In [ ]:
# Previous delirium
df['previous_delirium'] = df.groupby('subject_id')['delirium'].shift(1).fillna(0)

# Cumulative ICU admissions
df['cumulative_icu_admissions'] = df.groupby('subject_id')['icu_admission'].cumsum().shift(1).fillna(0)

# Cumulative major surgeries
df['cumulative_major_surgeries'] = df.groupby('subject_id')['major_surgery'].cumsum().shift(1).fillna(0)

# Time since last admission
df['time_since_last_admission'] = df.groupby('subject_id')['admittime'].diff().dt.days

# Total previous admissions
df['total_previous_admissions'] = df.groupby('subject_id').cumcount()

# Cumulative medication usage
df['cumulative_sedatives'] = df.groupby('subject_id')['sedatives'].cumsum().shift(1).fillna(0)
df['cumulative_anticholinergics'] = df.groupby('subject_id')['anticholinergics'].cumsum().shift(1).fillna(0)
df['cumulative_opioids'] = df.groupby('subject_id')['opioids'].cumsum().shift(1).fillna(0)

# Time since last admission
df['time_since_last_admission'].fillna(0, inplace=True)

In [ ]:
df

TypeError: Object of type Timestamp is not JSON serializable

       subject_id gender        dob           admittime           dischtime  \
55621           3      M 2025-04-11 2101-10-20 19:08:00 2101-10-31 13:58:00   
21558           6      F 2109-06-21 2175-05-30 07:15:00 2175-06-15 16:00:00   
54493          12      M 2032-03-24 2104-08-07 10:15:00 2104-08-20 02:57:00   
22728          20      F 2107-06-13 2183-04-28 09:45:00 2183-05-03 14:45:00   
50564          21      M 2047-04-04 2134-09-11 12:17:00 2134-09-24 16:15:00   
...           ...    ...        ...                 ...                 ...   
30779       99982      M 2091-10-02 2157-01-05 17:27:00 2157-01-12 13:00:00   
30781       99982      M 2091-10-02 2157-02-16 17:31:00 2157-02-22 20:36:00   
18602       99983      M 2114-09-29 2193-04-26 11:35:00 2193-04-29 13:30:00   
37015       99992      F 2078-10-17 2144-07-25 18:03:00 2144-07-28 17:56:00   
46077       99995      F 2058-05-29 2147-02-08 08:00:00 2147-02-11 13:15:00   

                   ethnicity cognitive_impairment_t

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Engineering
# Keep only the original 'time_since_last_admission' without scaling

# Step 2: Handle Missing Values
# Fill missing values only for relevant columns
for col in df.select_dtypes(include=['number']).columns:
    if pd.api.types.is_integer_dtype(df[col]):  # For integer columns
        df[col] = df[col].fillna(0).astype(int)  # Fill missing integers with 0
    else:  # For float columns
        df[col] = df[col].fillna(df[col].median())  # Fill missing floats with the median

# Step 3: Drop Irrelevant Columns
columns_to_drop = [
    'subject_id', 'dob', 'admittime', 'dischtime', 'age_group',
    'ethnicity'
]
df = df.drop(columns=columns_to_drop, axis=1)

# Step 4: One-Hot Encode Relevant Categorical Columns
# One-hot encode 'gender'
df = pd.get_dummies(df, columns=['gender'], drop_first=True)

# One-hot encode 'cognitive_impairment_type'
df = pd.get_dummies(df, columns=['cognitive_impairment_type'], prefix='cognitive_impairment', drop_first=True)


# Step 5: Split Features and Target
X = df.drop('delirium', axis=1)  # Features
y = df['delirium']  # Target

# Ensure all columns in X are numeric
# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object', 'category']).columns
if len(non_numeric_columns) > 0:
    print("Non-numeric columns found:", non_numeric_columns)

# Convert all columns to numeric (force invalid strings to NaN, then handle them)
X = X.apply(pd.to_numeric, errors='coerce')

# Check for any remaining NaN values and fill them with 0
X = X.fillna(0)

# Step 6: Handle Class Imbalance Using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the class distribution after resampling
print(y_resampled.value_counts())

delirium
0    22249
1    22249
Name: count, dtype: int64


In [ ]:
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 23480 entries, 55621 to 46077
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   is_deceased                    23480 non-null  int64  
 1   length_of_stay                 23480 non-null  float64
 2   sepsis                         23480 non-null  int64  
 3   heart_failure                  23480 non-null  int64  
 4   stroke                         23480 non-null  int64  
 5   icu_admission                  23480 non-null  int64  
 6   major_surgery                  23480 non-null  int64  
 7   delirium                       23480 non-null  int64  
 8   sedatives                      23480 non-null  int64  
 9   anticholinergics               23480 non-null  int64  
 10  opioids                        23480 non-null  int64  
 11  total_medications              23480 non-null  int64  
 12  fever                          23480 non-null  

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = df.drop('delirium', axis=1)  # Drop the target column
y = df['delirium']  # Target column

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(X_train.dtypes)
print(X_test.dtypes)


print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


is_deceased                        int64
length_of_stay                   float64
sepsis                             int64
heart_failure                      int64
stroke                             int64
icu_admission                      int64
major_surgery                      int64
sedatives                          int64
anticholinergics                   int64
opioids                            int64
total_medications                  int64
fever                              int64
hypotension                        int64
hyponatremia                       int64
hyperkalemia                       int64
infection                          int64
age_at_admission                   int64
previous_delirium                  int64
cumulative_icu_admissions          int64
cumulative_major_surgeries         int64
time_since_last_admission        float64
total_previous_admissions          int64
cumulative_sedatives               int64
cumulative_anticholinergics        int64
cumulative_opioi

In [ ]:
X_train

is_deceased  length_of_stay  sepsis  heart_failure  stroke  \
17613            1            17.0       0              0       0   
51216            0             3.0       0              1       0   
40139            1             1.0       0              0       0   
41489            0             9.0       0              0       0   
34613            1             1.0       0              0       0   
...            ...             ...     ...            ...     ...   
56134            0            13.0       0              0       0   
44342            0            45.0       1              0       0   
16524            1            14.0       0              1       0   
34798            1             6.0       0              0       0   
46672            1             4.0       0              0       0   

       icu_admission  major_surgery  sedatives  anticholinergics  opioids  \
17613              1              1          0                 0        0   
51216              1              0          0                 0        0   
40139              1              0          1                 0        0   
41489              1              0          0                 1        1   
34613              1              0          0                 0        0   
...              ...            ...        ...               ...      ...   
56134              1              0          1                 0        1   
44342              1              0          1                 1        1   
16524              1              0          1                 0        1   
34798              0              0          0                 0        0   
46672              1              0          1                 0        1   

       ...  cumulative_icu_admissions  cumulative_major_surgeries  \
17613  ...                          2                           2   
51216  ...                          2                           0   
40139  ...                          1                           1   
41489  ...                          1                           1   
34613  ...                          2                           0   
...    ...                        ...                         ...   
56134  ...                          1                           0   
44342  ...                          1                           0   
16524  ...                          1                           0   
34798  ...                          1                           0   
46672  ...                          1                           1   

       time_since_last_admission  total_previous_admissions  \
17613                        0.0                          0   
51216                        0.0                          0   
40139                        0.0                          0   
41489                        0.0                          0   
34613                       27.0                          2   
...                          ...                        ...   
56134                        0.0                          0   
44342                        0.0                          0   
16524                        0.0                          0   
34798                       67.0                          1   
46672                        0.0                          0   

       cumulative_sedatives  cumulative_anticholinergics  cumulative_opioids  \
17613                     0                            1                   2   
51216                     2                            1                   1   
40139                     0                            0                   1   
41489                     1                            0                   1   
34613                     0                            1                   0   
...                     ...                          ...                 ...   
56134                     0                            0                   1   
44342                     1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
print(f"Random Forest ROC-AUC Score: {roc_auc_rf:.4f}")


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4450
           1       1.00      0.00      0.01       246

    accuracy                           0.95      4696
   macro avg       0.97      0.50      0.49      4696
weighted avg       0.95      0.95      0.92      4696

Random Forest ROC-AUC Score: 0.6656


In [ ]:
from xgboost import XGBClassifier

# Initialize the XGBoost model
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]))

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
print(f"XGBoost ROC-AUC Score: {roc_auc_xgb:.4f}")


XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90      4450
           1       0.11      0.33      0.16       246

    accuracy                           0.82      4696
   macro avg       0.53      0.59      0.53      4696
weighted avg       0.91      0.82      0.86      4696

XGBoost ROC-AUC Score: 0.6498


In [ ]:
from lightgbm import LGBMClassifier

# Initialize the LightGBM model
lgbm_model = LGBMClassifier(random_state=42, class_weight='balanced')

# Train the model
lgbm_model.fit(X_train, y_train)

# Make predictions
y_pred_lgbm = lgbm_model.predict(X_test)
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("LightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgbm))

roc_auc_lgbm = roc_auc_score(y_test, y_pred_proba_lgbm)
print(f"LightGBM ROC-AUC Score: {roc_auc_lgbm:.4f}")


[LightGBM] [Info] Number of positive: 985, number of negative: 17799
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 577
[LightGBM] [Info] Number of data points in the train set: 18784, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.77      0.85      4450
           1       0.10      0.49      0.17       246

    accuracy                           0.75      4696
   macro avg       0.53      0.63      0.51      4696
weighted avg       0.92      0.75      0.82      4696

LightGBM ROC-AUC Score: 0.6876


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a pipeline with scaling and SVM
svm_model = make_pipeline(StandardScaler(), SVC(probability=True, class_weight='balanced', random_state=42))

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)
y_pred_proba_svm = svm_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))

roc_auc_svm = roc_auc_score(y_test, y_pred_proba_svm)
print(f"SVM ROC-AUC Score: {roc_auc_svm:.4f}")


SVM Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.72      0.83      4450
           1       0.09      0.52      0.16       246

    accuracy                           0.71      4696
   macro avg       0.53      0.62      0.49      4696
weighted avg       0.92      0.71      0.79      4696

SVM ROC-AUC Score: 0.6774
